<a href="https://colab.research.google.com/github/anqi-guo/anomaly_detection_isolation_forest/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

path = '/content/drive/My Drive/work/在线监测/data'
data = pd.read_csv(f'{path}/encoded_data.csv')
data.info(memory_usage=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4879543 entries, 0 to 4879542
Data columns (total 62 columns):
 #   Column           Dtype  
---  ------           -----  
 0   排口ID             int64  
 1   污染物ID            int64  
 2   污染物浓度值           float64
 3   污染物排放量           float64
 4   排放量              float64
 5   流量               float64
 6   污染物浓度上限值         float64
 7   是否生产             int64  
 8   生产状态_流量中位数       float64
 9   企业名称             int64  
 10  城市名称             int64  
 11  区县名称             int64  
 12  是否异常             int64  
 13  pol_diff1        float64
 14  pfl_diff1        float64
 15  ll_diff1         float64
 16  pol_diff2        float64
 17  pfl_diff2        float64
 18  ll_diff2         float64
 19  pol_diff3        float64
 20  pfl_diff3        float64
 21  ll_diff3         float64
 22  pol_pctchg1      float64
 23  pfl

In [23]:
for c in data.select_dtypes(include=['float64']).columns:
  data[c] = data[c].astype('float32')

for c in data.select_dtypes(include=['int64']).columns:
  data[c] = data[c].astype('int32')

data.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4879543 entries, 0 to 4879542
Data columns (total 62 columns):
 #   Column           Dtype  
---  ------           -----  
 0   排口ID             int32  
 1   污染物ID            int32  
 2   污染物浓度值           float32
 3   污染物排放量           float32
 4   排放量              float32
 5   流量               float32
 6   污染物浓度上限值         float32
 7   是否生产             int32  
 8   生产状态_流量中位数       float32
 9   企业名称             int32  
 10  城市名称             int32  
 11  区县名称             int32  
 12  是否异常             int32  
 13  pol_diff1        float32
 14  pfl_diff1        float32
 15  ll_diff1         float32
 16  pol_diff2        float32
 17  pfl_diff2        float32
 18  ll_diff2         float32
 19  pol_diff3        float32
 20  pfl_diff3        float32
 21  ll_diff3         float32
 22  pol_pctchg1      float32
 23  pfl_pctchg1      float32
 24  ll_pctchg1       float32
 25  pol_pctchg2      float32
 26  pfl_pctchg2      float32
 27  ll_pctchg2  

In [24]:
import numpy as np

data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(method='bfill', inplace=True)

In [26]:
data.isnull().any()

排口ID              False
污染物ID             False
污染物浓度值            False
污染物排放量            False
排放量               False
                  ...  
ll_pctchg_lag3    False
year              False
month             False
day               False
hour              False
Length: 62, dtype: bool

In [27]:
from sklearn.ensemble import IsolationForest

model=IsolationForest(n_estimators=100,max_samples='auto',contamination=float(0.2),random_state=0)

model.fit(data)

print(model.get_params())

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


{'bootstrap': False, 'contamination': 0.2, 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 100, 'n_jobs': None, 'random_state': 0, 'verbose': 0, 'warm_start': False}


In [28]:
data['year'].unique()

array([2021, 2022], dtype=int32)

In [34]:
data_test = data.query('year==2022 and month==4')

In [35]:
data_test['anomaly_score'] = model.predict(data_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
from datetime import datetime

data_ = data_test.query('排口ID==0')
data_['监测时间'] = data_.apply(lambda row: datetime(int(row['year']), int(row['month']), int(row['day']), int(row['hour'])), axis=1)

data_['anomaly_score'] = data_['anomaly_score'].astype(int) 
data_['是否异常'] = data_['是否异常'].astype(int) 
data_['different_abnormal'] = np.where(data_['是否异常']>data_['anomaly_score'], 1, 0)

for c in ['anomaly_score', '是否异常', 'different_abnormal']:
  data_[c] = data_[c].astype('str')

import plotly.express as px
fig = px.scatter(data_frame=data_, x='监测时间', y='污染物浓度值', color='anomaly_score', facet_row='污染物ID')
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [65]:
fig = px.scatter(data_frame=data_, x='监测时间', y='污染物浓度值', color='是否异常', facet_row='污染物ID')
fig.show()

In [66]:
fig = px.scatter(data_frame=data_, x='监测时间', y='污染物浓度值', color='different_abnormal', facet_row='污染物ID')
fig.show()